In [1]:
import numpy as np 
import pandas as pd 

###################### Entrainement boîte noire 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import string 
from nltk import word_tokenize
os.chdir("C:/Users/33651/Documents/Projet_Anchors/CODE/")

################ Mon implémentation 
from perturbation import *  
import itertools
import copy
from selection_meilleur_ancre_DETERMINISTE import * 

###############PACKAGE 
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys


In [2]:
from chargement_donnée import * 

Val accuracy 0.8


# Performance de la nouvelle implémentation 
### Expérience sur 1000 commentaires (Perturbation déterministe)

In [7]:

## Fonction qui renvoie la décision prise par le classifieur.
def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

#PACKAGE
nlp = spacy.load('en_core_web_sm')
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=True)

np.random.seed(1)

# Récupération des commentaires 
sen = df["sentence"]
sen_a = np.array(sen)

## On prend des phrases pas trop grande pour éviter que ce soit trop long 
## car on realise une recherche exhaustive sur 1000 commentaires si on choisit des commentaires trop long
## le temps de calcul risque d'être trop long (commentaire long = nombre combinaison très grande)
phrase_expl = []
for i in range(len(sen_a)):
    if len(sen_a[i]) <= 35: 
        phrase_expl.append(sen_a[i])
    

np.random.seed(1)
list_performance =[]
perf = 0
prec = []
res= 0
etap = 1
phrase_l= []
ancre_faux_p = []
ancre_faux_moi = []


while perf < 10 : # On réalise 10 fois les mêmes étapes (explication de 100 commentaires avec le package et notre implémentation)
                  # et comparaison des deux sorties.
    g=0
    indice_phrase =[]
    etap = 0
    print("étape :", perf,"/10")
    # On choisit 100 commentaires aléatoirement - Pour éviter de récupérer des commentaires mal rédiger ou n'ayant pas de sens,
    # on realise un nettoyage en testant si la boîte noire reussit à trouver l'ancre avec une grande précision. 
    # Si le commentaire était fausser, la décision prise sur les perturbations ne seraient pas stable 
    # On aurait donc une précision faible; (étape pas obligatoire). 
    
    while g < 100 :
        precis = 0
        while precis != 1 : 
            indice=  np.random.choice(len(phrase_expl), 1 ,replace=False)
           # print(indice)
           # print(phrase_expl[indice[0]])
            
            exp3 = explainer.explain_instance(phrase_expl[indice[0]], predict_lr, threshold=0.95)
            
            precis = exp3.precision()
            
        if indice[0] not in indice_phrase :
            indice_phrase.append(indice[0])
            g=g+1
    
    
    res = 0 # Variable qui va permettre de compter le nombre de fois où on obtient le même résultat.
    for i in indice_phrase: # Boucle qui va de la phrase 0 à la phase 100 (pour chaque phrase application du package + nvlle impl)
      #  print(phrase_expl[i])
        label = predict_lr([phrase_expl[i]])[0] #Décision prise par le classifieur pour la phrase étudier ()
        
    
      #  print("Le label associé est", label)
        
        
     #   print("La phrase est", phrase_expl[i])
    # phrase contient toutes les phrases expliquées.
        phrase_l.append(phrase_expl[i])
        # Mon implémentation - recherche de la meilleure ancre par méthode exhaustive.
        moi = meilleur_ancre_bis_new(df,phrase_expl[i],250,"UNK",label,0.15,vectorizer,c)
        #récupération du nom de l'ancre trouvé
        moi_nom = moi[0]
        #récupération de la précision de l'ancre trouvée
        moi_prec = moi[1]
        #Application du package - recherche de la meilleure ancre par méthode bandit manchot
        exp3 = explainer.explain_instance(phrase_expl[i], predict_lr, threshold=0.95)
        #récupération du nom de l'ancre trouvé
        ancre = exp3.names()
        #récupération de la précision de l'ancre trouvée
        precis = exp3.precision()
        
        # Pour éviter de trouver 2 ancres différents à cause de majuscule / minuscule 
        # exemple :(Love != love) alors que les deux mots sont identiques.
        
        for k in range(len(ancre)):
            ancre[k] = ancre[k].lower()
        
        #Si l'ancre est composée de plusieurs mots, pour comparer les deux listes on range les mots dans le même ordre
        # exemple : A1 = [book, reception] , A2 = [reception , book ] 
        # sorted => A1 = [book, reception] , A2 = [book, reception] (ordre alphabétique)
        ancre_moi = sorted(moi_nom)
        ancre_pack = sorted(ancre)
        #print("COMPARAISON : MON ANCRE ", ancre_moi ,"VS" , ancre_pack )
         
        #On regarde a present si les deux listes sont identiques
        # => Si oui on compte +1
        if ancre_moi == ancre_pack : 
            
            res = res + 1 
            #print("Résultat identique")
            #print("Précision obtenue avec mon implémentation ", moi_prec )
           # print('Precision_p: %.2f' % exp3.precision())
        # => Sinon on l'ajoute dans une liste pour qu'on puisse analyser les erreurs.   
        else : 
            prec.append(precis)
            ancre_faux_p.append(ancre_pack)
            ancre_faux_moi.append(ancre_moi)
           # print("Résultat différent")
           # print("Précision obtenue avec mon implémentation ", moi_prec )
          #  print('Precision_p: %.2f' % exp3.precision())
    
        
        etap = etap+1
    
    perf = perf +1 
    list_performance.append(res)

étape : 0 /10
étape : 1 /10
étape : 2 /10
étape : 3 /10
étape : 4 /10
étape : 5 /10
étape : 6 /10
étape : 7 /10
étape : 8 /10
étape : 9 /10


In [8]:
df_perf = pd.DataFrame(list_performance,columns = ["Performance nouvelle implémentation"])

In [9]:
df_perf

,Performance nouvelle implémentation
0,91
1,98
2,95
3,99
4,95
5,96
6,94
7,93
8,92
9,96


In [10]:
performance_moyenne = np.mean(list_performance)

In [11]:
print("La performance moyenne est de ", performance_moyenne)

La performance moyenne est de  94.9


Regardons les erreurs : 

In [12]:

df = pd.DataFrame(list(zip(ancre_faux_p,ancre_faux_moi)), columns = ['Ancre package','Ancre nouvelle implémentation'])
    
df

,Ancre package,Ancre nouvelle implémentation
0,[ ],[.]
1,[me],[work]
2,"[favorite, one]","[favorite, purchases]"
3,[do],[dont]
4,[ ],[.]
5,[ ],[.]
6,[ ],[.]
7,[ ],[.]
8,[ ],[!]
9,[ ],[!]


###### On constate que globalement les erreurs sont dû à un mauvais traitement de commentaire ou à un commentaire non préparer ; erroné.
Par exemple : 
  - Mauvaise décompostion des mots de la phrase : "10+ VS 10" ; "check VS service-chek" 
  
  - Commentaire non préparer (erroné) :  "[!]" VS "[]" ou "[.]" (une simple ponctuation ne peut pas être important dans la prise de décision du classifieur concernant le sentiment transmit par ce commentaire).
  
  - L'algorithme d'optimisation du package est différent de l'algorithme utilisé dans notre implémentation : 
      - Implémentation : Recherche exhaustive.
      - Package : Bandit Manchot.
      
  - Aléa de la génèration des pertuabations. Suivant le bruit ajouté, la décision prise par le classifieur peut différer, et donc modifier légèrement la valeur de la précision. 